# Amazon Bedrock AgentCore Runtime と AgentCore Memory エージェント

## 概要

このチュートリアルでは、AgentCore Runtime と AgentCore Memory を使用して、初めての Memory 対応エージェントを作成する方法を説明します。セッション内で以前のインタラクションを記憶するシンプルな「Hello World」会話エージェントを構築します。

### チュートリアル詳細


| 項目              | 詳細                                                             |
|:------------------|:-----------------------------------------------------------------|
| チュートリアルタイプ | Hello World / 入門                                              |
| エージェントタイプ   | 単一会話エージェント                                              |
| エージェントフレームワーク | Strands Agents                                              |
| LLM モデル         | Anthropic Claude Haiku 4.5                                      |
| 主な機能           | AgentCore Runtime、Memory 統合                                   |
| 難易度             | 初級                                                             |
| 使用 SDK          | boto3、bedrock-agentcore、bedrock-agentcore-starter-toolkit      |

### 学習内容

このチュートリアルでは、以下を学習します：
1. エージェント用の Memory リソースを作成する方法
2. 会話履歴を保存・取得するための Memory フックを実装する方法
3. エージェントを AgentCore Runtime にデプロイする方法
4. セッション管理を使用してエージェントをテストする方法


### アーキテクチャ

この Hello World の例は、Memory 統合を備えた AgentCore Runtime にデプロイされたシンプルな会話エージェントを示しています：

<div style="text-align:left">
    <img src="RuntimeMemoryIntegration.png" width="90%"/>
</div>

## 0. 前提条件

このチュートリアルを実行するには、以下が必要です：
* Python 3.10 以上
* AWS 認証情報の設定
* Amazon Bedrock モデルアクセス（Claude 3.7 Sonnet）
* Amazon Bedrock AgentCore SDK

まず、必要なライブラリをインストールしましょう：

In [ ]:
!pip install -qr requirements.txt

### 環境のセットアップ

必要なライブラリをインポートし、環境を設定しましょう：

In [ ]:
# Imports
import os
import boto3
import uuid
import logging
from bedrock_agentcore.memory import MemoryClient

# Configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(name)s: %(message)s'
)
logger = logging.getLogger("runtime-memory-agent")
REGION = os.getenv('AWS_REGION', 'us-west-2') # AWS region for the agent
memory_client = MemoryClient(region_name=REGION)

## 1. Memory リソースの作成

このセクションでは、エージェントが会話履歴を保存するための Memory リソースを作成します。Memory により、エージェントは過去のインタラクションを思い出し、コンテキストを維持し、時間の経過とともにより一貫性のある応答を提供できます。

この例では、追加の長期ストラテジーを持たないシンプルな短期 Memory リソースを作成します。Memory はすべての会話メッセージを保存し、AgentCore Runtime でセッションが終了した後も会話を継続する際にエージェントが以前のインタラクションを記憶するのに役立ちます。

In [ ]:
from botocore.exceptions import ClientError

# Create unique identifier for this resource
unique_id = str(uuid.uuid4())[:8]
memory_name = f"RuntimeMemoryAgent_{unique_id}"

try:
    # Create memory resource without strategies (short-term memory only)
    memory = memory_client.create_memory_and_wait(
        name=memory_name,
        strategies=[],  # No strategies for short-term memory
        description="Short-term memory for AgentCore Runtime agent",
        event_expiry_days=7, # Retention period for short-term memory
    )
    memory_id = memory['id']
    logger.info(f"✅ Created memory: {memory_id}")
except ClientError as e:
    logger.info(f"❌ ERROR: {e}")
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # If memory already exists, retrieve its ID
        memories = memory_client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"Memory already exists. Using existing memory ID: {memory_id}")
except Exception as e:
    # Show any errors during memory creation
    logger.error(f"❌ ERROR: {e}")
    import traceback
    traceback.print_exc()
    # Cleanup on error - delete the memory if it was partially created
    if 'memory_id' in locals() and memory_id:
        try:
            memory_client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"Cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.error(f"Failed to clean up memory: {cleanup_error}")

## 2. Memory 対応エージェントの作成

このセクションでは、Memory 統合用のカスタムフックを備えた Strands Agents フレームワークを使用して、Memory 対応エージェントを構築します。このエージェントは、AgentCore Memory からメッセージを保存・取得することで会話コンテキストを維持します。

> **Memory が重要な理由**：AgentCore Runtime のセッションは一定時間後に期限切れになり、会話コンテキストが削除されます。会話を Memory に保存することで、セッション間で以前の情報が永続化され、長い休憩の後でもユーザーにシームレスな体験を提供できます。

### エージェントの機能

エージェントは以下を行います：
1. 各ユーザーメッセージとアシスタントメッセージを自動的に Memory に保存
2. 既存のセッションを継続する際に過去の会話履歴を取得
3. 同じユーザーとの複数のインタラクションにわたってコンテキストを維持

### 実装の主要コンポーネント

#### 1. Memory フックプロバイダー
カスタムフックプロバイダーは以下を実装します：
- `on_agent_initialized`：エージェント開始時にトリガーされ、会話履歴を取得
- `on_message_added`：新しいメッセージが会話に追加されたときにトリガー

#### 2. エージェント初期化
`initialize_agent` 関数：
- 正しいリージョンで Memory フックを設定
- 適切な状態変数とシステムプロンプトでエージェントを設定

#### 3. エントリポイントハンドラー
runtime_memory_agent 関数：
- 入力ペイロードを解析し、ユーザーメッセージを抽出
- エージェントの初期化とセッション追跡を管理
- 適切なコンテキストでエージェントの呼び出しを処理
- Runtime 環境にフォーマットされた応答を返す

エージェントファイルを作成しましょう：

In [ ]:
%%writefile runtime_memory_agent.py
import os
import json
import logging
from typing import Dict, Any
from strands import Agent
from strands.models import BedrockModel
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands.hooks import AgentInitializedEvent, HookProvider, HookRegistry, MessageAddedEvent

# Configure detailed logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(name)s: %(message)s'
)
logger = logging.getLogger("runtime-memory-agent")

# Initialize the agent core app
app = BedrockAgentCoreApp()

MODEL_ID = os.getenv('MODEL_ID')
MEMORY_ID = os.getenv('MEMORY_ID')
REGION = os.getenv('AWS_REGION')

# Global agent instance - will be initialized with first request
agent = None

class MemoryHookProvider(HookProvider):
    """Custom hook provider to integrate with Bedrock Memory"""
    
    def __init__(self, region_name):
        logger.info(f"Initializing MemoryHookProvider with region {region_name}")
        self.memory_client = MemoryClient(region_name=region_name)
    
    def on_agent_initialized(self, event: AgentInitializedEvent):
        """Load recent conversation history when agent starts"""
        logger.info("Agent initialization hook triggered")
        
        memory_id = event.agent.state.get("memory_id")
        actor_id = event.agent.state.get("actor_id")
        session_id = event.agent.state.get("session_id")
        
        logger.info(f"State values - memory_id: {memory_id}, actor_id: {actor_id}, session_id: {session_id}")
        
        missing_values = []
        if not memory_id:
            missing_values.append("memory_id")
        if not actor_id:
            missing_values.append("actor_id")
        if not session_id:
            missing_values.append("session_id")
            
        if missing_values:
            logger.warning(f"Missing required values: {', '.join(missing_values)}")
            return
        
        try:
            # First, check if the session exists by listing events with a limit of 1
            logger.info(f"Checking if session {session_id} exists...")
            session_exists = False
            try:
                events = self.memory_client.list_events(
                    memory_id=memory_id,
                    actor_id=actor_id,
                    session_id=session_id,
                    max_results=1
                )
                session_exists = len(events) > 0
                logger.info(f"Session exists: {session_exists} (found {len(events)} events)")
            except Exception as e:
                logger.warning(f"Error checking session existence: {e}")
                # Assume no events exist and continue
                session_exists = False
            
            # If session doesn't exist, no need to load conversation history
            if not session_exists:
                logger.info(f"No existing conversation found for session {session_id}")
                return
            
            # Session exists, load the conversation history
            logger.info(f"Loading conversation history for existing session {session_id}")
            recent_turns = self.memory_client.get_last_k_turns(
                memory_id=memory_id,
                actor_id=actor_id,
                session_id=session_id,
                k=5
            )
            
            if recent_turns:
                logger.info(f"✅ Loaded {len(recent_turns)} conversation turns from memory")
                context_messages = []
                for turn in recent_turns:
                    for message in turn:
                        role = message['role']
                        content = message['content']['text']
                        context_messages.append(f"{role}: {content}")
                
                context = "\n".join(context_messages)
                event.agent.system_prompt += f"\n\nRecent conversation:\n{context}"
                logger.info("✅ Added conversation context to system prompt")
            else:
                logger.info("No recent turns found for this session")
                
        except Exception as e:
            logger.error(f"❌ Memory load error: {e}", exc_info=True)
    
    def on_message_added(self, event: MessageAddedEvent):
        """Store messages in memory"""
        logger.info("Message added hook triggered")
        
        memory_id = event.agent.state.get("memory_id")
        actor_id = event.agent.state.get("actor_id")
        session_id = event.agent.state.get("session_id")
        
        logger.info(f"State values - memory_id: {memory_id}, actor_id: {actor_id}, session_id: {session_id}")
        
        missing_values = []
        if not memory_id:
            missing_values.append("memory_id")
        if not actor_id:
            missing_values.append("actor_id")
        if not session_id:
            missing_values.append("session_id")
            
        if missing_values:
            logger.warning(f"❌ Cannot save message - missing values: {', '.join(missing_values)}")
            return
            
        messages = event.agent.messages
        try:
            last_message = messages[-1]
            message_content = str(last_message.get("content", ""))
            message_role = last_message["role"]
            
            logger.info(f"Saving {message_role} message to memory: {message_content[:30]}...")
            
            self.memory_client.create_event(
                memory_id=memory_id,
                actor_id=actor_id,
                session_id=session_id,
                messages=[(message_content, message_role)]
            )
            logger.info("✅ Message saved to memory successfully")
        except Exception as e:
            logger.error(f"❌ Memory save error: {e}", exc_info=True)
    
    def register_hooks(self, registry: HookRegistry):
        logger.info("Registering memory hooks")
        registry.add_callback(MessageAddedEvent, self.on_message_added)
        registry.add_callback(AgentInitializedEvent, self.on_agent_initialized)

def initialize_agent(actor_id, session_id):
    """Initialize the agent for first use"""
    global agent
    
    logger.info(f"Initializing agent for actor_id={actor_id}, session_id={session_id}")
    
    # Create model and memory hook
    logger.info(f"Creating model with ID: {MODEL_ID}")
    model = BedrockModel(model_id=MODEL_ID)
    logger.info(f"Creating memory hook with region: {REGION}")
    memory_hook = MemoryHookProvider(region_name=REGION)
    
    # Create agent with proper initial state
    logger.info("Creating agent with memory hook")
    agent = Agent(
        model=model,
        hooks=[memory_hook],
        system_prompt="You're a helpful, memory-enabled agent deployed on AgentCore Runtime. You can remember previous interactions within the same session. Be friendly and concise in your responses.",
        state={
            "memory_id": MEMORY_ID,
            "actor_id": actor_id,
            "session_id": session_id
        }
    )
    logger.info(f"✅ Agent initialized with state: {agent.state.get()}")

@app.entrypoint
def runtime_memory_agent(payload, context):
    """
    Main entry point for the memory-enabled agent
    
    Args:
        payload: The input payload containing user data
        context: The runtime context object containing session information
    """
    global agent
    
    # Log both payload and context info
    logger.info(f"Received payload: {payload}")
    logger.info(f"Context session_id: {context.session_id}")
    
    # Extract and validate required values
    user_input = payload.get("prompt")
    actor_id = payload.get("actor_id", "default_user")  # Provide default for demo
    session_id = context.session_id  # Get session_id from context
    
    # Validate required fields
    if user_input is None:
        error_msg = "❌ ERROR: Missing 'prompt' field in payload"
        logger.error(error_msg)
        return error_msg
    
    # Initialize agent on first request
    if agent is None:
        logger.info("First request - initializing agent")
        initialize_agent(actor_id, session_id)
    else:
        logger.info("Using existing agent instance")
        # Update the session ID in case it changed
        if agent.state.get("session_id") != session_id:
            logger.info(f"Updating session ID to {session_id}")
            agent.state.set("session_id", session_id)
        if agent.state.get("actor_id") != actor_id:
            logger.info(f"Updating actor ID to {actor_id}")
            agent.state.set("actor_id", actor_id)
    
    # Invoke the agent with the user's input
    logger.info(f"Invoking agent with input: {user_input}")
    response = agent(user_input)
    response_text = response.message['content'][0]['text']
    logger.info(f"✅ Agent response: {response_text[:50]}...")
    
    return response_text

if __name__ == "__main__":
    logger.info("Starting AgentCore application")
    app.run()

## 3. AgentCore Runtime へのデプロイ

このセクションでは、エージェントを Amazon Bedrock AgentCore Runtime（スケーラビリティと運用の簡素化を提供するマネージドエージェントランタイム環境）にデプロイします。AgentCore Runtime はインフラストラクチャの複雑さを処理するため、デプロイメントの懸念ではなくエージェントのロジックに集中できます。

手動のサーバーセットアップと管理を必要とする従来のデプロイメント方法とは異なり、AgentCore Runtime はコードを自動的にコンテナにパッケージ化し、AWS インフラストラクチャにデプロイし、呼び出し用の安全な HTTPS エンドポイントを提供します。このアプローチにより、エージェントは需要に応じてスケールし、本番環境で確実に動作できます。

### 知っておくべきこと

- **AgentCore Runtime** はエージェントを Docker コンテナにパッケージ化し、マネージド AWS インフラストラクチャにデプロイします
- **環境変数** でエージェントを設定します：
  - `MEMORY_ID`：先ほど作成した Memory リソース
  - `MODEL_ID`：Claude 3.7 Sonnet モデル ID
  - `AWS_REGION`：デプロイ用の AWS リージョン

> 💡 **ヒント**：AgentCore スターターツールキットは、IAM ロール、ECR リポジトリ、コンテナビルドなど、すべての複雑なデプロイメント手順を処理します。

### デプロイメントの設定

デプロイメント設定をセットアップしましょう：

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
import time

agentcore_runtime = Runtime()
agent_name = f"runtime_memory_agent_{unique_id}"

response = agentcore_runtime.configure(
    entrypoint="runtime_memory_agent.py", 
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=REGION,
    agent_name=agent_name
)
response

### エージェントの起動

AgentCore Runtime にエージェントを起動しましょう。このステップでは、設定したエージェントを取得し、AgentCore のマネージドインフラストラクチャにデプロイします。このプロセス中に、エージェントに必要な基本的な環境変数（先ほど作成した Memory ID と使用するモデル ID）も渡しています。デプロイ後、エージェントはユーザーメッセージで呼び出せるセキュアなエンドポイントを通じてアクセス可能になります。

In [ ]:
launch_result = agentcore_runtime.launch(
    env_vars={
        "MEMORY_ID": memory_id,
        "MODEL_ID": "global.anthropic.claude-haiku-4-5-20251001-v1:0",
    }
)

### デプロイメントステータスの確認

エージェントのデプロイメントステータスを確認しましょう：

In [ ]:
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']

while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(f"Current status: {status}")

if status == 'READY':
    print("✅ Agent successfully deployed!")
else:
    print(f"❌ Deployment ended with status: {status}")

## エージェントのテスト

エージェントがデプロイされたので、メッセージを送信してテストしましょう。

**Actor ID とセッション管理に関する重要な注意事項**

- Actor ID：本番アプリケーションでは、actor_id は通常、ユーザーがログインしたときに認証システムから取得されます。この識別子により、エージェントは異なるユーザーに対して個別の会話履歴を維持できます。この例では、ハードコードされた値（test_user_123）を使用していますが、実際のシナリオでは、認証されたユーザーの一意の識別子を渡すことになります。

- セッション管理：AgentCore Runtime はセッション ID が提供されない場合に自動的に生成しますが、アプリケーションでセッション ID を明示的に管理することをお勧めします。これにより、以下をより適切に制御できます：
  - セッションタイムアウト後の会話の継続
  - 適切な場合（例：ユーザーが新しい会話を開始）に新しいセッションを作成
  - 同じユーザーとの複数の並行会話の処理
  - アプリケーションのニーズに基づいたセッション有効期限ポリシーの実装

In [ ]:
# Generate a test session ID
test_session_id = "agent-runtime-memory-session-123456789" # Min length is 33

# Send our first message
invoke_response = agentcore_runtime.invoke(
    {
        "prompt": "Hello! My name is John. What can you do?",
        "actor_id": "test_user_123"
    }, 
    session_id=test_session_id
)

invoke_response

### エージェントの応答を表示

より読みやすい形式で応答を表示しましょう：

In [ ]:
from IPython.display import Markdown, display
import json

response_text = invoke_response['response'][0]
display(Markdown(response_text))

### 永続性のテスト

エージェントが以前のインタラクションを記憶しているかどうかをテストするために、同じセッションでフォローアップメッセージを送信しましょう：

In [ ]:
# Send a follow-up message using the same session ID
follow_up_response = agentcore_runtime.invoke(
    {
        "prompt": "What is my name?",
        "actor_id": "test_user_123"
    }, 
    session_id=test_session_id
)

# Display the response
follow_up_text = follow_up_response['response'][0]
display(Markdown(follow_up_text))

### Memory 内容の確認

メッセージが正しく保存されていることを確認するために、Memory に保存されている内容を確認しましょう：

In [ ]:
memory_client = MemoryClient(region_name=REGION)
stored_events = memory_client.list_events(
    memory_id=memory_id,
    actor_id="test_user_123", 
    session_id=test_session_id
)

# Reverse the list to show oldest events first
stored_events.reverse()

print(f"Found {len(stored_events)} events in memory (shown in chronological order):")
for idx, event in enumerate(stored_events):
    print(f"\nEvent {idx+1}:")
    # The structure is different - messages are in the payload.conversational
    for payload_item in event['payload']:
        if 'conversational' in payload_item:
            role = payload_item['conversational']['role']
            text = payload_item['conversational']['content']['text']
            # Clean up the text that contains string representations of dictionaries
            if text.startswith("[{") and text.endswith("}]"):
                # Extract just the actual message text
                import re
                clean_text = re.search(r"'text': '(.*?)'", text)
                if clean_text:
                    text = clean_text.group(1)
            print(f"- {role}: {text[:100]}...")

## 主要な概念

このチュートリアルでは、以下を学習しました：

1. **Memory 統合**：Amazon Bedrock Memory を使用して会話履歴を保存する方法
2. **セッション管理**：セッション ID を使用して会話コンテキストを維持する方法
3. **AgentCore デプロイメント**：エージェントを本番ランタイム環境にデプロイする方法
4. **Memory フック**：Memory サービスとやり取りするためのカスタムフックを実装する方法

これらの概念は、永続的な Memory を持つより複雑なエージェントを構築するための基盤を提供します。

## クリーンアップ（オプション）

このチュートリアルで作成したリソースが不要になった場合は、クリーンアップできます：

In [ ]:
# Get resource identifiers
if 'launch_result' in locals():
    print(f"Agent ID: {launch_result.agent_id}")
    print(f"ECR Repository: {launch_result.ecr_uri.split('/')[1]}")
else:
    print("Launch results not available")

In [ ]:
# Only run this cell if you want to delete the resources

# Delete the AgentCore Runtime
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=REGION
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
)
print(f"Deleted AgentCore Runtime: {launch_result.agent_id}")

# Delete the ECR repository
ecr_client = boto3.client(
    'ecr',
    region_name=REGION
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)
print(f"Deleted ECR repository: {launch_result.ecr_uri.split('/')[1]}")

# Delete the memory resource
memory_client = MemoryClient(region_name=REGION)
memory_client.delete_memory_and_wait(memory_id=memory_id)
print(f"Deleted memory resource: {memory_id}")

## おめでとうございます！

Amazon Bedrock AgentCore Runtime と AgentCore Memory を使用した初めての Memory 対応エージェントの構築とデプロイに成功しました！

### 次のステップ

基本を理解したので、以下のことができます：

1. **ツールの追加**：計算機、データベースコネクタ、API 呼び出しなどのツールでエージェントを強化
2. **Memory の改善**：長期 Memory を使用したより洗練された Memory ストラテジーを実装
3. **UI の構築**：エージェント用の Web またはモバイルインターフェースを作成